In [ ]:
import numpy as np
import pandas as pd 
from keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import random
import os

In [ ]:
flights = pd.read_excel("D:/Suyog's Docs/Data/FlightData/AmericanAIData.xlsx")#,header=None, names=flight_headers, dtype=flight_dtypes) #, header =[1])
flights.head(5)

In [ ]:
def get_count_percentage (df, col_name):
    dfcount = df.groupby(col_name)
    counts = dfcount.count()
    percentage = round(dfcount.count().mean() * 100, 1)
    percentages = percentage.sort_values()
    null_values = pd.concat([counts, percentages], axis=1, keys=["counts", "%"])
    return counts.iloc[:,1]

In [ ]:
def get_cols_name(df, lit):
    feature_col_name = df[[lit]]
    col_name = feature_col_name.columns.values
    return col_name[0]

In [ ]:
American_Al = flights.loc[flights["DEST_AIRPORT"] == 'Seattle-Tacoma International Airport']
American_Al.head(5)

In [ ]:
#Convert timestamp to proper value
def insertChar(mystring ):
    lenofstring = len(mystring)
    if (lenofstring == 1):
        mystring = '0' + mystring[0:] +':' + '00'
    elif (lenofstring == 2):
        if (int(mystring[0:2]) <= 24):
            mystring = mystring[:] + ':00'
        else :
            mystring =  '00:' +mystring 
    elif (lenofstring == 3):
        if (int(mystring[0:2]) <= 24):
            mystring = mystring[0:2] + ':' + mystring[2:3] + '0'
        else:    
            mystring = '0' + mystring[1:] + ':00'
    elif (lenofstring > 4):
        mystring = mystring[0:4]
        mystring = mystring[0:2] + ':' + mystring[2:] 
    elif (lenofstring == 4):
        mystring = mystring[0:2] + ':' + mystring[2:]
    return mystring 

In [ ]:
#convert timestamps into proper format
American_Al["TIMESTAMP"] = pd.to_datetime(American_Al['YEAR']+ American_Al['MONTH']+ American_Al['DAY']).dt.date

In [ ]:
American_Al["DEPARTURE_TIME"] = American_Al["DEPARTURE_TIME"].astype(str).apply(insertChar)
American_Al["SCHEDULED_DEPARTURE"] =American_Al["SCHEDULED_DEPARTURE"].astype(str).apply(insertChar)
American_Al["SCHEDULED_ARRIVAL"] =American_Al["SCHEDULED_ARRIVAL"].astype(str).apply(insertChar)
American_Al["ARRIVAL_TIME"] =American_Al["ARRIVAL_TIME"].astype(str).apply(insertChar)

In [ ]:
American_Al.columns

In [ ]:
print(American_Al.columns)

In [ ]:
#get monthwise count for simplicity we will consider only two months data
col_name = get_cols_name(American_Al,'MONTH')
print(get_count_percentage(American_Al, col_name))
American = American_Al.loc[(American_Al['MONTH'] == 7) | (American_Al['MONTH'] == 8)]
American_Al.head(5)
#American_Al = pd.DataFrame(American_Al) #.to_frame().reset_index()
print(get_count_percentage(American, col_name))

In [ ]:
#For simplicti we wil take onlu Aug and July data
American = flights.loc[flights["DEST_AIRPORT"] == 'Seattle-Tacoma International Airport']
American.head(5)

In [ ]:
American["MONTHDAY"] = American["MONTH"].astype(str) +  American["DAY"].astype(str)


In [ ]:
#plot the month day wise graph to check if delyas are higher
sns.set()
sns.set(rc={'figure.figsize':(100,40)})
plot = sns.barplot(x='MONTHDAY',y='ARRIVAL_DELAY',data=American)
colors = sns.color_palette('bright')[0:5]
plot.set_title("Daywise  delay")

In [ ]:
#Plot histrogram to determin delay intervalswith bin size of 5 mins
sns.set()
sns.set(rc={'figure.figsize':(30,20)})
#histogram_data = sns.load_dataset("American")
#sns.histplot(data=histogram_data , x="ARRIVAL_DELAY", bins=10)
American["ARRIVAL_DELAY"].hist(bins=10)
plt.xlabel("Arrival Delays in mins", fontsize=40)
plt.ylabel("Frequency",fontsize=40)

In [ ]:
# From Histogram, we can say the target data is not balanced, most of the time there is no delay or before arrival time. 
# let su get percentile of the arrival delays to understand pattern, also we can convert the data into three categoreis
# Before time (arrival delay < 0), on time ( arrival delay < 10 mins)  and delayed (arrival dealy > 10 mins). We can calculate
#entropy of the data using above categories
# get arrival edlay and days from the dataframe
arrival_delays = American[["ARRIVAL_DELAY","MONTHDAY"]]
arrival_delays.head(5)

In [ ]:
def get_percentile(df,col,percentile_keys):
    counts ={}
    i = 0 
    print(col)
    percentile = df[[col]].quantile(percentile_keys)
    #create list on datframe 
    percentages = percentile[col].tolist()
    for percent in percentages:
        count = (df[df[col] < percent].shape[0])
        counts[percentile_keys[i]] = count
        i = i+1
    return counts

In [ ]:
percentile_keys = [0.25, 0.90]
#we need to get value counts below 25 % abd above 95% percentile to see how extrem delays are there 
# get first ranges  0f 25% and 95%
percentile = arrival_delays[["ARRIVAL_DELAY"]].quantile(percentile_keys)
percent = arrival_delays["ARRIVAL_DELAY"].tolist()
counts = {} 
counts['below 0.25'] = (arrival_delays[arrival_delays.ARRIVAL_DELAY < percent[0]].shape[0])
counts ['moderate delays'] = (arrival_delays[(arrival_delays.ARRIVAL_DELAY > percent[0]) & (arrival_delays.ARRIVAL_DELAY < percent[1])].shape[0])
counts ['extreme delays'] = (arrival_delays[arrival_delays.ARRIVAL_DELAY > percent[1]].shape[0])
print(counts)
#print(arrival_delays[arrival_delays.ARRIVAL_DELAY < -5.0].shape[0])
# we will drop extreme delays as the outlier will overfit the model
print(percentile)




In [ ]:
American = American[(American.ARRIVAL_DELAY < 37) & (American.ARRIVAL_DELAY > -15)]
American.describe()

In [ ]:
# AS dataframe consist of both numerical and categorical data, only continous data needs to be scaled / normalized. 
#Below function will use standard deviation method to normalize column data
# input column name and dataframe
def normalize_cols(df, col_name): 
    df[col_name] -= df[col_name].mean()
    df[col_name] /= df[col_name].std()
    return df
    

In [ ]:
#convert the select cols and normalize them
List_of_cols = ["DEPARTURE_DELAY", "TAXI_OUT", "WHEELS_OFF", "SCHEDULED_TIME", "ELAPSED_TIME", "AIR_TIME", "WHEELS_ON", "TAXI_IN", "ARRIVAL_DELAY"] 
for cols in List_of_cols:
    American[cols] = normalize_cols(American[[cols]].astype(float),cols)
American.head(5)

In [ ]:
# From above graph we can see there are days where delays are extreme. This will lead to overfitting of the model. 
#plot them as box plot. Before that we need to normalize data
american_airline_outliers = American[["MONTHDAY" ,"DEPARTURE_DELAY", "TAXI_OUT", "WHEELS_OFF", "ELAPSED_TIME", "AIR_TIME", "WHEELS_ON", "TAXI_IN", "ARRIVAL_DELAY"]].astype(float)
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
col_names = american_airline_outliers.columns
d = scaler.fit_transform(american_airline_outliers)
scaled_data = pd.DataFrame(d, columns=["DEPARTURE_DELAY", "TAXI_OUT", "WHEELS_OFF", "ELAPSED_TIME", "AIR_TIME", "WHEELS_ON", "TAXI_IN", "ARRIVAL_DELAY"])
scaled_data.head()


In [ ]:
#Create another dataframe and plot graph
American.head()
#american_ariline_outliers = American_Al.loc[American_Al["DEPARTURE_DELAY"], American_Al["TAXI_OUT"], American_Al["WHEELS_OFF"], American_Al["ELAPSED_TIME"], American_Al ["AIR_TIME"], American_Al["WHEELS_ON"], American_Al["TAXI_IN"],American_Al["ARRIVAL_DELAY"]]

#american_airline_outliers.head()
#figure, axes = plt.subplots(2,4, sharex=True, figsize=(10,20))
colors = sns.color_palette('bright')[0:5]
sns.set(rc={'figure.figsize':(40,25)})
#scaled_data.boxplot(column=["DEPARTURE_DELAY", "TAXI_OUT", "WHEELS_OFF", "ELAPSED_TIME"],by=["MONTHDAY"], layout=(2,2),fontsize=20)
American.boxplot(column=["DEPARTURE_DELAY", "TAXI_OUT", "WHEELS_OFF", "ELAPSED_TIME"],by=["MONTHDAY"],layout=(2,2),fontsize=20)
                                          

#ax_1 = american_airline_outliers.boxplot(column=["AIR_TIME", "WHEELS_ON", "TAXI_IN", "ARRIVAL_DELAY"])

In [ ]:
colors = sns.color_palette('bright')[0:5]
sns.set(rc={'figure.figsize':(30,25)})
#scaled_data.boxplot(column=["DEPARTURE_DELAY", "TAXI_OUT", "WHEELS_OFF", "ELAPSED_TIME"],by=["MONTHDAY"], layout=(2,2),fontsize=20)
American.boxplot(column=["AIR_TIME", "WHEELS_ON", "TAXI_IN", "ARRIVAL_DELAY"],by=["MONTHDAY"],layout=(2,2),fontsize=20)

In [ ]:
#plot correlation and covarinace between Taxiin, out departure delay, arrival  delay elapsed time and airtime to see if any relation
Amreican_features = American[["DEPARTURE_DELAY", "TAXI_OUT", "WHEELS_OFF", "SCHEDULED_TIME", "ELAPSED_TIME","AIR_TIME", "WHEELS_ON", "TAXI_IN", "ARRIVAL_DELAY"]]


In [ ]:
def plot_corr(df,size=10):
    """Function plots a graphical correlation matrix for each pair of columns in the dataframe.

    Input:
        df: pandas DataFrame
        size: vertical and horizontal size of the plot
    """

    corr = df.corr()
    fig, ax = plt.subplots(figsize=(size, size))
    #ax.matshow(corr)
    #plt.xticks(range(len(corr.columns)), corr.columns)
    #plt.yticks(range(len(corr.columns)), corr.columns)
    sns.heatmap(corr,annot=True, mask=np.zeros_like(corr,dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax)

In [ ]:
plot_corr(Amreican_features)

In [ ]:
#plot correlation matrix- pairplot 
sns.pairplot(Amreican_features, hue ='ARRIVAL_DELAY')

In [ ]:
#plot covariance matrix
def correlated_cols(dataframe,threshold):
    col_corr = set()
    corr_matrix = dataframe.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range (i):
            if abs(corr_matrix.iloc[i,j]) > threshold:
                colname = corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

In [ ]:
columns_to_drop = correlated_cols(Amreican_features,0.9)
print(columns_to_drop)
American.columns

In [ ]:
#Dropping column from the dataset having high correlation to avoid the overfitting 
Features =  American.drop(columns_to_drop,axis=1)



In [ ]:
Features.head(5)

In [ ]:
#plot the covariance of the numerical data
Amreican_features.cov()

In [ ]:
Amreican_features.corr()

In [ ]:
#Features.head(5)
def get_nun_unique_count(df,colname):
    #n = df.colname.nunique()
    n = df[colname].nunique()
    return n

In [ ]:
cols_to_drop = set()
for col in Features.columns:
    if (get_nun_unique_count(Features,col) < 10):
        cols_to_drop.add(col)
#print(cols_to_drop)
Features_Selected = Features.drop(cols_to_drop,axis=1)


In [ ]:
Features_Selected.dtypes

In [ ]:
#let us convert the arrival time and departure time into the 4 bins mid night(00:00 - 06:00), morning (06:00-12:00), afternoon(12:00-18:00), night (18:00-24:00)
bins = [ 600,1200,1800,2400,np.inf]
names = ['night' ,'earlymorning', 'morning' ,'afternoon']
Features_Selected['DEPARTURE_TIME'] = pd.cut(Features_Selected['DEPARTURE_TIME'], bins, labels=names)


In [ ]:
Features_Selected['ARRIVAL_TIME'] = pd.cut(Features_Selected['ARRIVAL_TIME'], bins, labels=names)
Features_Selected.head()

In [ ]:
#Let us drop Month, day,  SCHEDULED_DEPARTURE, SCHEDULED_ARRIVAL and MONTHDAY coloumn as those are not significant
American_al_features = Features_Selected.drop(['SCHEDULED_DEPARTURE', 'SCHEDULED_ARRIVAL', 'MONTH', 'DAY', 'MONTHDAY'],axis=1)
American_al_features.head(5)

In [ ]:
#check if flight_number and tail number for unique value. Looks liie they will be repeating for each ferry and do not make any significance
print("Flight Number Unique records", American_al_features['FLIGHT_NUMBER'].nunique())
print("TAIL Number Unique records" , American_al_features['TAIL_NUMBER'].nunique())
print("Total Counts" , American_al_features.count())

In [ ]:
# As the unique values are less compare to total counts, the flight numebrs and tail numbers are repeating everytime. This wont be useful and needs to 
# drop further.
American_al_data =  American_al_features.drop(['FLIGHT_NUMBER','TAIL_NUMBER'],axis=1)
American_al_data.head(5)
#DEPARTURE_TIME and ARRIVAL_TIME  are the categorical value which needs to be converted into numerical data. We need to apply one-hot encoding
dummy_departure_time = pd.get_dummies(American_al_data.DEPARTURE_TIME)
dummy_arrival_time = pd.get_dummies(American_al_data.ARRIVAL_TIME)
# add suffix to dummy col name to avoid dupaice columns
dummy_departure_time = dummy_departure_time.add_prefix('departure_time_')
dummy_arrival_time = dummy_arrival_time.add_prefix('arrival_time_')
#join newly created departure and arrival time 
American_dl_dummy_departure = pd.concat([American_al_data,dummy_departure_time],axis='columns')
#American_dl_dummy_departure.head()
American_dl_dummy_arrival = pd.concat([American_dl_dummy_departure,dummy_arrival_time],axis='columns')
American_dl_dummy_arrival.head()
#let us drop orginal catgory column from the Df
America_Al_Features = American_dl_dummy_arrival.drop(['DEPARTURE_TIME','ARRIVAL_TIME'],axis=1 )
#to avoid dummy variable trap or multi colinearity , we need to drop one columns from the dummy variables created 
#dropping deparutre_time_night and arrival_time_night from the dataframe
America_Al_Features = American_dl_dummy_arrival.drop(['DEPARTURE_TIME','ARRIVAL_TIME','departure_time_night','arrival_time_night'],axis=1 )
America_Al_Features.head()

In [ ]:
American_Al_data = America_Al_Features
American_Al_data.head(5)

In [ ]:
#As we have converted and selected all the features that can be apply to ML model, we need to idenify most important features
#this can be achiveved using information gain
#get dependent variable


y_lable = America_Al_Features.pop('ARRIVAL_DELAY')
x_lable = America_Al_Features

#import required libaraires
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
# feature selection
f_selector = SelectKBest(score_func=f_regression, k='all')
# learn relationship from training data
f_selector.fit(x_lable, y_lable)
# transform train input data
X_train_fs = f_selector.transform(x_lable)
# Plot the scores for the features
plt.bar([i for i in range(len(f_selector.scores_))], f_selector.scores_)
plt.xlabel("feature index")
plt.ylabel("F-value (transformed from the correlation values)")
plt.show()




In [ ]:
American_Al_data.head()

In [ ]:
#From above , thecolumn # 0,1,2,3,5,7,9 and 10 have significant imapct and we will drop column 4,6 from the dataframe and save the same.
# column 4, 6, 8 are wheeels_on, departure_time_earlymorning, arrival_time_earlymorning and arrival_time_afternoon . we will drop wheels_on.
Flight_delay_prediction_features = American_Al_data.drop(['WHEELS_ON'],axis=1)
Flight_delay_prediction_features.head(5)
Flight_delay_prediction_features.to_csv("D:/Suyog's Docs/Data/FlightData/features.csv")
